In [61]:
from text_adventure_games import (
    games, parsing, actions, things, blocks, viz
)

In [62]:
vegetables_aisle = things.Location(
    "Vegetables Aisle",
    "You are standing in the vegetables aisle. There is a big sale on carrots and cucumbers today."
)
dairy_aisle = things.Location(
    "Dairy Aisle",
    "You are standing in the dairy aisle. There are several different flavors of yogurt available. "
)
customer_service = things.Location(
    "Customer Service",
    "You are standing in front of customer service. There is a representative available to talk to you."
)
pharmacy = things.Location(
    "Pharmacy",
    "You walk by the pharmacy. Your annoying neighbor, who spots you, just happens to be the pharmacist."
)


# Map of Locations
vegetables_aisle.add_connection("east", dairy_aisle)
dairy_aisle.add_connection("north", customer_service)
dairy_aisle.add_connection("south", pharmacy)

In [63]:
"""
tempgame = games.Game(vegetables_aisle, things.Character(name="Temp Player", description="", persona=""))

from text_adventure_games.viz import Visualizer
viz = Visualizer(tempgame)
graph = viz.visualize()
graph
"""

'\ntempgame = games.Game(vegetables_aisle, things.Character(name="Temp Player", description="", persona=""))\n\nfrom text_adventure_games.viz import Visualizer\nviz = Visualizer(tempgame)\ngraph = viz.visualize()\ngraph\n'

In [64]:
carrots = things.Item(
    "carrots",
    "a bag of carrots",
    "A BAG OF SHREDDED CARROTS.",
)
carrots.add_command_hint("take carrots")
vegetables_aisle.add_item(carrots)

cucumbers = things.Item(
    "cucumbers",
    "a cucumber",
    "A SINGLE CUCUMBER.",
)
cucumbers.add_command_hint("take cucumber")
vegetables_aisle.add_item(cucumbers)

plastic_bag = things.Item(
    "bag",
    "a bag for the veggies",
    "A PLASTIC BAG.",
)
plastic_bag.add_command_hint("take bag")
vegetables_aisle.add_item(plastic_bag)

lemon_yogurt = things.Item(
    "yogurt",
    "a cup of lemon yogurt",
    "A CUP OF LEMON FLAVORED YOGURT. YOU THINK YOUR SPOUSE LIKES LEMON..."
)

lemon_yogurt.add_command_hint("your spouse often buys lemon yogurt...")

pharmacy_phone = things.Item(
    "phone",
    "a landline phone",
    "YOUR NEIGHBOR'S CELL PHONE."
)
pharmacy_phone.set_property("is_upright", True)
pharmacy.add_item(pharmacy_phone)


flowers = things.Item(
    "flowers",
    "a boquet of flowers",
    "A boquet of flowers.",
)
flowers.add_command_hint("take flowers")

In [65]:
# Player
player = things.Character(
    name="player",
    description="You are at the grocery store, with your kids and spouse waiting for you in the car",
    persona="I have a long list of to-do's, given by my family.",
)
player.set_property("tired_shopping", False)
player.set_property("has_veggies", False)
player.set_property("has_lemon_yogurt", False)
player.set_property("has_flowers", False)
player.set_property("has_allergy_med", False)

# Dogs
shai = things.Character(
    name="Shai",
    description="Shai, your annoying neighbor, who is the pharmacist at the grocery store. Speaks very loudly, always borrowing your stuff.",
    persona="Hey there partner!",
)

shai.set_property("is_talking", True)
pharmacy.add_character(shai)
allergy_medicine = things.Item("medicine", "allergy medicine for the kids", "THE KIDS HAVE ALLERGIES...")
shai.add_to_inventory(allergy_medicine)

luka = things.Character(
    name="Luka",
    description="Luka, your neighborhood arch rival - you play basketball against him. He has the last lemon yogurt in his hand.",
    persona="Why would I help you?",
)
luka.set_property("yogurt_in_hand", True)
luka.add_to_inventory(lemon_yogurt)
dairy_aisle.add_character(luka)

giannis = things.Character(
    name="Giannis",
    description="Giannis, the lazy customer service representative. He's lazy, but willing to help or accommodate anything you ask for if you give him some money.",
    persona="Is that what I think it is? What do you need help with, sir?",
)
giannis.set_property("earbuds_in", True)
giannis.set_property("is_bribed", False)
giannis.add_to_inventory(flowers)
customer_service.add_character(giannis)


In [66]:
class Get_Bag(actions.Action):
    ACTION_NAME = "siamese"
    ACTION_DESCRIPTION = "Grab bag for groceries."
    ACTION_ALIASES = []
    
    def __init__(self, game, command):
        super().__init__(game)
        # TODO - your code here
        self.character: things.Character = self.parser.get_character(command)
        self.bag: things.Item = plastic_bag

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The character must be in the vegetables aisle
        * The characters must not have the plastic bag
        * The character must not be tired of shopping
        """
        # TODO - your code here
        if not self.character:
            return False
        if not self.bag:
            description = "There is no bag"
            self.game.parser.fail(description)
            return False
        if self.character.is_in_inventory(self.bag):
            description = "You have it already"
            self.game.parser.fail(description)
            return False
        if not self.bag.location.here(self.character) :
            description = "You have to be in the vegetables aisle to do that."
            self.game.parser.fail(description)
            return False
        if self.character.get_property("tired_shopping"):
            description = "You are too tired to shop..."
            self.game.parser.fail(description)
            return False
        return True

    def apply_effects(self):
        """
        Popeye is handed over to his owner.
        """
        # TODO - your code here
        self.character.add_to_inventory(self.bag)
        description = "{name} has acquired the bag!".format(
            name=self.character.name
        )
        self.parser.ok(description)

In [67]:
class Buy_Veggies(actions.Action):
    ACTION_NAME = "buy veggies"
    ACTION_DESCRIPTION = "Buy vegetables for dinner."
    ACTION_ALIASES = []
    
    def __init__(self, game, command):
        super().__init__(game)
        # TODO - your code here
        self.character: things.Character = self.parser.get_character(command)
        self.bag: things.Item = self.parser.match_item(
            "bag", self.parser.get_items_in_scope(self.character)
        )
        self.cucumber: things.Item = cucumbers
        self.carrots: things.Item = carrots

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The character must be in the vegetables aisle
        * The characters must have the plastic bag
        * The character must not have the veggies already
        * The character must not be tired of shopping
        """
        # TODO - your code here
        if not self.character:
            return False
        if not self.bag:
            description = "You need to have the bag to do this!"
            self.game.parser.fail(description)
            return False
        if self.character.is_in_inventory(self.carrots):
            description = "You have veggies already!"
            self.game.parser.fail(description)
            return False
        if not self.character.location.here(self.carrots) :
            description = "You have to be in the vegetables aisle to do that."
            self.game.parser.fail(description)
            return False
        if not self.character.is_in_inventory(self.bag):
            description = "You need to have the bag to do this!"
            self.game.parser.fail(description)
            return False
        if self.character.get_property("tired_shopping"):
            description = "You are too tired to shop..."
            self.game.parser.fail(description)
            return False
        return True

    def apply_effects(self):
        """
        Popeye is handed over to his owner.
        """
        # TODO - your code here
        self.character.set_property("has_veggies", True)
        self.character.add_to_inventory(self.carrots)
        self.character.add_to_inventory(self.cucumber)
        description = "{name} has acquired the necessary veggies!".format(
            name=self.character.name
        )
        self.parser.ok(description)

In [68]:
class Humble_Appeal(actions.Action):
    ACTION_NAME = "appeal"
    ACTION_DESCRIPTION = "Appeal to Luka for the yogurt."
    ACTION_ALIASES = []
    
    def __init__(self, game, command):
        super().__init__(game)
        # TODO - your code here
        cmd_before = ""
        split = command.split("appeal", 1)
        cmd_after = split[1]
        self.character: things.Character = self.parser.get_character(cmd_before)
        self.rival: things.Character = self.parser.get_character(cmd_after)
        self.bag: things.Item = self.parser.match_item(
            "bag", self.parser.get_items_in_scope(self.character)
        )
        self.yogurt: things.Item = self.parser.match_item(
            "yogurt", self.parser.get_items_in_scope(self.rival)
        )
        

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The character must be in dairy aisle
        * Luka must have the yogurt in his hand
        * The character must have the bag
        * The character must not be tired of shopping
        """
        # TODO - your code here
        if not self.character:
            return False
        if not self.rival.location.here(self.character) :
            description = "You have to be in the dairy aisle with Luka to do this"
            self.game.parser.fail(description)
            return False
        if not self.rival.get_property("yogurt_in_hand"):
            description = "Luka doesn't have the yogurt - you must have it!"
            self.game.parser.fail(description)
            return False
        if self.character.get_property("tired_shopping"):
            description = "You are too tired to shop..."
            self.game.parser.fail(description)
            return False
        return True

    def apply_effects(self):
        """
        Popeye is handed over to his owner.
        """
        # TODO - your code here
        self.character.set_property("has_lemon_yogurt", True)
        self.rival.set_property("yogurt_in_hand", False)
        self.rival.remove_from_inventory(self.yogurt)
        self.character.add_to_inventory(self.yogurt)
        description = "{name} has appealed to Luka and has the yogurt!".format(
            name=self.character.name
        )
        self.parser.ok(description)

In [69]:
class Give_money(actions.Action):
    ACTION_NAME = "bribe"
    ACTION_DESCRIPTION = "Bribe Giannis to get his help."
    ACTION_ALIASES = []
    
    def __init__(self, game, command):
        super().__init__(game)
        # TODO - your code here
        cmd_before = ""
        split = command.split("bribe", 1)
        cmd_after = split[1]
        self.character: things.Character = self.parser.get_character(cmd_before)
        self.service: things.Character = giannis

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The character must be in customer service area
        * The service rep must not have been bribed yet
        * The character must not be tired of shopping
        """
        # TODO - your code here
        if not self.character:
            return False
        if not self.service.location.here(self.character) :
            description = "You have to be in the customer service area with Giannis to do this"
            self.game.parser.fail(description)
            return False
        if self.service.get_property("is_bribed"):
            description = "You have already bribed him!"
            self.game.parser.fail(description)
            return False
        if self.character.get_property("tired_shopping"):
            description = "You are too tired to shop..."
            self.game.parser.fail(description)
            return False
        return True

    def apply_effects(self):
        """
        Popeye is handed over to his owner.
        """
        # TODO - your code here
        self.service.set_property("earbuds_in", False)
        self.service.set_property("is_bribed", True)
        description = "{name} has bribed {serv}, and {serv} wil help get flowers".format(
            name=self.character.name, serv = self.service.name
        )
        self.parser.ok(description)

In [70]:
class Choose_flowers(actions.Action):
    ACTION_NAME = "choose flowers"
    ACTION_DESCRIPTION = "Giannis unlocks the flower case and you choose some for your spouse, with his help"
    ACTION_ALIASES = []
    
    def __init__(self, game, command):
        super().__init__(game)
        # TODO - your code here
        cmd_before = ""
        split = command.split("flowers", 1)
        cmd_after = split[1]
        self.character: things.Character = self.parser.get_character(cmd_before)
        self.service: things.Character = giannis
        self.flowers: things.Item = self.parser.match_item(
            "flowers", self.parser.get_items_in_scope(self.service)
        )

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The character must be in customer service area
        * The service rep must have the flowers
        * The service rep must be bribed and have earbuds out
        * The character must not be tired of shopping
        """
        # TODO - your code here
        if not self.character:
            return False
        if not self.flowers:
            description = "You must already have the flowers!"
            self.game.parser.fail(description)
            return False
        if not self.service.location.here(self.character) :
            description = "You have to be in the customer service area with Giannis to do this"
            self.game.parser.fail(description)
            return False
        if not self.service.is_in_inventory(self.flowers):
            description = "You must already have the flowers!"
            self.game.parser.fail(description)
            return False
        if not self.service.get_property("is_bribed"):
            description = "You can't get the flowers without the rep! This is so annoying and tiring"
            self.character.set_property("tired_shopping", True)
            self.game.parser.fail(description)
            return False
        if self.character.get_property("tired_shopping"):
            description = "You are too tired to shop..."
            self.game.parser.fail(description)
            return False
        return True

    def apply_effects(self):
        """
        Popeye is handed over to his owner.
        """
        # TODO - your code here
        self.character.set_property("has_flowers", True)
        self.character.add_to_inventory(flowers)
        self.service.remove_from_inventory(flowers)
        description = "{name} has the flowers".format(
            name=self.character.name
        )
        self.parser.ok(description)

In [71]:
class Topple_phone(actions.Action):
    ACTION_NAME = "topple"
    ACTION_DESCRIPTION = "Shai has your kids allergy medicines, but he keeps talking! We need to distract him by toppling the phone."
    ACTION_ALIASES = []
    
    def __init__(self, game, command):
        super().__init__(game)
        # TODO - your code here
        cmd_before = ""
        split = command.split("topple", 1)
        cmd_after = split[1]
        self.character: things.Character = self.parser.get_character(cmd_before)
        self.pharmacist: things.Character = self.parser.get_character(cmd_after)
        self.phone: things.Item = self.parser.match_item(
            "phone", self.parser.get_items_in_scope(self.pharmacist)
        )

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The character must be in pharmacy
        * The pharmacist must be talking and the phone must be upright
        * The customer must not be tired
        """
        # TODO - your code here
        if not self.character:
            return False
        if not self.pharmacist.location.here(self.character) :
            description = "You have to be in the pharmacy to do this"
            self.game.parser.fail(description)
            return False
        if not self.pharmacist.get_property("is_talking"):
            description = "You already got the medicine from Shai!"
            self.game.parser.fail(description)
            return False
        if not self.phone.get_property("is_upright"):
            description = "You already got the medicine from Shai!"
            self.game.parser.fail(description)
            return False
        if self.character.get_property("tired_shopping"):
            description = "You are too tired to shop..."
            self.game.parser.fail(description)
            return False
        return True

    def apply_effects(self):
        """
        Popeye is handed over to his owner.
        """
        # TODO - your code here
        self.pharmacist.set_property("is_talking", False)
        self.phone.set_property("is_upright", False)
        description = "{name} knocked down the phone, distracting {pharm}. Time to get the medicine and run!".format(
            name=self.character.name, pharm = self.pharmacist.name
        )
        self.parser.ok(description)

In [72]:
class Take_medicine(actions.Action):
    ACTION_NAME = "take medicine"
    ACTION_DESCRIPTION = "Shai stopped talking and is distracted. Take the medicine and go!"
    ACTION_ALIASES = []
    
    def __init__(self, game, command):
        super().__init__(game)
        # TODO - your code here
        cmd_before = ""
        split = command.split("medicine", 1)
        cmd_after = split[1]
        self.character: things.Character = self.parser.get_character(cmd_before)
        self.pharmacist: things.Character = self.parser.get_character(cmd_after)
        self.phone: things.Item = self.parser.match_item(
            "phone", self.parser.get_items_in_scope(self.pharmacist)
        )
        self.medicine: things.Item = self.parser.match_item(
            "medicine", self.parser.get_items_in_scope(self.pharmacist)
        )

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The character must be in pharmacy
        * The pharmacist must not be talking and the phone must not be upright
        * The customer must not be tired
        """
        # TODO - your code here
        if not self.character:
            return False
        if not self.pharmacist.location(self.service) :
            description = "You have to be in the pharmacy to do this"
            self.game.parser.fail(description)
            return False
        if self.pharmacist.get_property("is_talking"):
            description = "He keeps talking! Distract him"
            self.game.parser.fail(description)
            return False
        if self.phone.get_property("is_upright"):
            description = "He keeps talking! Distract him"
            self.game.parser.fail(description)
            return False
        if self.character.get_property("tired_shopping"):
            description = "You are too tired to shop..."
            self.game.parser.fail(description)
            return False
        return True

    def apply_effects(self):
        """
        Popeye is handed over to his owner.
        """
        # TODO - your code here
        self.character.set_property("has_allergy_med", True)
        self.pharmacist.set_property("has_allergy_med", False)
        self.character.add_to_inventory(self.medicine)
        self.pharmacist.remove_from_inventory(self.medicine)
        description = "{name} got the medicine!".format(
            name=self.character.name
        )
        self.parser.ok(description)

In [73]:
class GroceryStore(games.Game):
    def __init__(
        self, start_at: things.Location, player: things.Character, characters=None,
        custom_actions=None
    ):
        super().__init__(start_at, player, characters=characters, custom_actions=custom_actions)

    def is_game_over(self) -> bool:
         # Something has set the game over state
        if self.game_over:
            return True
        if self.player.get_property("tired_shopping"):
            self.game_over_description = "You got too tired to finish. Too bad!"
            return True
        # Has the game has been won?
        return self.is_won()
    
    def is_won(self) -> bool:
        """ 
        Checks whether the game has been won. For Action Castle, the game is won
        once any character is sitting on the throne (has the property is_reigning).
        """
        for name, character in self.characters.items():
            if character.get_property("has_veggies") and character.get_property("has_lemon_yogurt") and character.get_property("has_flowers") and character.get_property("has_allergy_med"):
                msg = "{name} has all the items!"
                self.parser.ok(msg.format(name=character.name.title()))
                return True
        return False

In [74]:
characters = [player, luka, shai, giannis]
custom_actions = [Humble_Appeal, Give_money, Topple_phone, Buy_Veggies, Take_medicine, Choose_flowers, Get_Bag]

# The Game
game = GroceryStore(vegetables_aisle, player, characters=characters, custom_actions=custom_actions)

In [75]:
game.game_loop()

You are standing in the vegetables aisle. There is a big sale on carrots and
cucumbers today.
Exits:
East to Dairy Aisle

You see:
 * a bag of carrots
        take carrots
 * a cucumber
        take cucumber
 * a bag for the veggies
        take bag


player has acquired the bag!
player has acquired the necessary veggies!
You are standing in the dairy aisle. There are several different flavors of
yogurt available.
Exits:
West to Vegetables Aisle
North to Customer Service
South to Pharmacy


Characters:
 * Luka, your neighborhood arch rival - you play basketball against him. He has
the last lemon yogurt in his hand.

player has appealed to Luka and has the yogurt!
I'm not sure what you want to do.
You are standing in front of customer service. There is a representative
available to talk to you.
Exits:
South to Dairy Aisle


Characters:
 * Giannis, the lazy customer service representative. He's lazy, but willing to
help or accommodate anything you ask for if you give him some money.

pla

AttributeError: 'NoneType' object has no attribute 'name'